In [4]:
import os
import matplotlib.pyplot as plt
import mxnet as mx
from mxnet.gluon import nn
from gluoncv.utils import viz
from mxnet import nd

In [31]:
'''
ConvBlock
[Conv2d ─ BN ─ ACTIVATION]
'''
def ConvBlock(channels, kernel_size, strides, padding, use_bias=False, activation='leaky'):
    block = nn.HybridSequential()
    block.add(nn.Conv2D(int(channels), kernel_size=kernel_size, strides=strides, padding=padding, use_bias=use_bias))

    if not use_bias:
        block.add(nn.BatchNorm(in_channels=int(channels)))

    if activation == 'leaky':
        block.add(nn.LeakyReLU(0.1))

    return block


'''
RedidualBlock
[   ┌───────────────────────┐    ]
[In ┴ Conv(1x1) ─ Conv(3x3) ┴ out]
'''
class ResidualBlock(nn.HybridBlock):
    def __init__(self, channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = ConvBlock(channels / 2, kernel_size=1, strides=1, padding=0)
        self.conv2 = ConvBlock(channels, kernel_size=3, strides=1, padding=1)

    def hybrid_forward(self, F, x):       
        block = self.conv1(x)        
        block = self.conv2(block)        
        out = block + x

        return out

In [40]:
class DarkNet(nn.HybridBlock):
    def __init__(self, num_classes=1000, input_size=416):
        super(DarkNet, self).__init__()
        self.layer_num = 0
        self.num_classes = num_classes
        self.input_size = input_size

        self.input_layer = nn.Conv2D(channels=32, kernel_size=3, strides=1, padding=1, use_bias=False)

        self.layer1 = self.make_layer(64, 1)
        self.layer2 = self.make_layer(128, 2)
        self.layer3 = self.make_layer(256, 8)
        self.layer4 = self.make_layer(512, 8)
        self.layer5 = self.make_layer(1024, 4)

        self.global_avg_pool = nn.GlobalAvgPool2D()
        self.fc = nn.Dense(self.num_classes)

    def hybrid_forward(self, F, x):
        x = self.input_layer(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.global_avg_pool(x)
        x = self.fc(x)

        return x

    def make_layer(self, channels, layer_size=1):
        layer = nn.HybridSequential()

        layer.add(ConvBlock(channels, kernel_size=3, strides=2, padding=1))

        for i in range(layer_size):
            layer.add(ResidualBlock(channels))

        return layer

In [41]:
net = ResidualBlock(64)
net.initialize()
x = nd.random.normal(shape=(1, 64, 128, 128))
y = net(x)
print('y: ', y.shape, type(y))

y:  (1, 64, 128, 128) <class 'mxnet.ndarray.ndarray.NDArray'>


In [42]:
conv0 = ConvBlock(32, kernel_size=3, strides=2, padding=1)
conv0.initialize()
x = nd.random.normal(shape=(1, 32, 128, 128))
y = conv0(x)
print('y: ', y.shape, type(y))

y:  (1, 32, 64, 64) <class 'mxnet.ndarray.ndarray.NDArray'>


In [43]:
net = DarkNet(num_classes=5, input_size=416)
net.initialize()
x = nd.random.normal(shape=(1, 3, 416, 416))
y = net(x)
print('y: ', y.shape, type(y))

y:  (1, 5) <class 'mxnet.ndarray.ndarray.NDArray'>


In [44]:
net.summary(x)

--------------------------------------------------------------------------------
        Layer (type)                                Output Shape         Param #
               Input                            (1, 3, 416, 416)               0
            Conv2D-1                           (1, 32, 416, 416)             864
            Conv2D-2                           (1, 64, 208, 208)           18432
         BatchNorm-3                           (1, 64, 208, 208)             256
         LeakyReLU-4                           (1, 64, 208, 208)               0
            Conv2D-5                           (1, 32, 208, 208)            2048
         BatchNorm-6                           (1, 32, 208, 208)             128
         LeakyReLU-7                           (1, 32, 208, 208)               0
            Conv2D-8                           (1, 64, 208, 208)           18432
         BatchNorm-9                           (1, 64, 208, 208)             256
        LeakyReLU-10        